# 2B

In [141]:
import joblib
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectFromModel, SelectKBest
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.linear_model import Lasso
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
import sklearn.preprocessing as preprocessing

In [142]:
random_state = 42

In [143]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from pandas._libs.parsers import k
from sklearn import preprocessing, metrics
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold

In [144]:
mvp_stats_2b = pd.read_csv("datasets/player_mvp_stats.csv", ';')
nicknames_df = pd.read_csv("datasets/nicknames.csv", ';')
salaries_2b = pd.read_csv("datasets/salaries.csv")
teams_2b = pd.read_csv("datasets/teams.csv", ';')
us_inflation_2b = pd.read_csv("datasets/US CPI.csv")

C:\Users\imman\AppData\Local\Temp\ipykernel_5548\3205523321.py:1: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  mvp_stats_2b = pd.read_csv("datasets/player_mvp_stats.csv", ';')
C:\Users\imman\AppData\Local\Temp\ipykernel_5548\3205523321.py:2: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  nicknames_df = pd.read_csv("datasets/nicknames.csv", ';')
C:\Users\imman\AppData\Local\Temp\ipykernel_5548\3205523321.py:4: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  teams_2b = pd.read_csv("datasets/teams.csv", ';')


### 2B. Preprocess dan merge beberapa dataset menjadi satu

In [145]:
def dataframe_to_dict(nicknames_df, key_column_name='Name', value_column_name='Abbreviation'):
    keys = nicknames_df[key_column_name]
    values = nicknames_df[value_column_name]
    return dict(zip(keys, values))

In [146]:
def cleaning_teamsdf_removeUnimportantRows(teams_2b):
    teams_2b = teams_2b.copy()
    important_rows = ~teams_2b.W.str.contains("Division")
    return teams_2b[important_rows]

def cleaning_teamsdf_convertStripToNan(teams_2b):
    teams_2b = teams_2b.copy()
    teams_2b.GB = teams_2b.GB.replace("—", np.NAN)
    return teams_2b

def cleaning_teamsdf_convertToAppropriateDatatype(teams_2b):
    """
    preconditions:
        cleaning_teamsdf_removeUnimportantRows
         cleaning_teamsdf_convertStripToNan
    """
    teams_2b = teams_2b.copy()
    teams_2b.W = pd.to_numeric(teams_2b.W)
    teams_2b.L = pd.to_numeric(teams_2b.L)
    teams_2b['W/L%'] = pd.to_numeric(teams_2b['W/L%'])
    teams_2b.GB = pd.to_numeric(teams_2b.GB)
    teams_2b['PS/G'] = pd.to_numeric(teams_2b['PS/G'])
    teams_2b['PA/G'] = pd.to_numeric(teams_2b['PA/G'])
    teams_2b.SRS = pd.to_numeric(teams_2b.SRS)
    return teams_2b

def cleaning_anyDf_removeStarInTeamName(teams_2b, column_name='Team'):
    teams_2b = teams_2b.copy()
    teams_2b[column_name] = teams_2b[column_name].str.replace('*', '')
    return teams_2b


def cleaning_anyDf_convertTeamNamesToItsAbbrev(teams_2b, nicknames_df, column_name='Team'):
    """
    precondition:
        teams df:
            cleaning_anyDf_removeStarInTeamName
    """
    nicknames_df = nicknames_df.copy()
    teams_2b = teams_2b.copy()

    nicknames_df = nicknames_df.drop_duplicates(subset='Name', keep='first')
    mapping = dataframe_to_dict(nicknames_df)
    mapping |= dataframe_to_dict(nicknames_df, 'Abbreviation', 'Abbreviation')

    teams_2b[column_name] = list(map(
        lambda x: mapping[x],
        teams_2b[column_name]
    ))
    return teams_2b

In [147]:
teams_2b = cleaning_teamsdf_removeUnimportantRows(teams_2b)
teams_2b = cleaning_teamsdf_convertStripToNan(teams_2b)
teams_2b = cleaning_teamsdf_convertToAppropriateDatatype(teams_2b)
teams_2b = cleaning_anyDf_removeStarInTeamName(teams_2b)
teams_2b = cleaning_anyDf_convertTeamNamesToItsAbbrev(teams_2b, nicknames_df)
teams_2b

C:\Users\imman\AppData\Local\Temp\ipykernel_5548\1280597476.py:29: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  teams_2b[column_name] = teams_2b[column_name].str.replace('*', '')


,W,L,W/L%,GB,PS/G,PA/G,SRS,Year,Team
0,56,26,683.0,NaN,111.5,105.7,5.22,1991,BOS
1,44,38,537.0,12.0,105.4,105.6,-0.39,1991,PHI
2,39,43,476.0,17.0,103.1,103.3,-0.43,1991,NYK
3,30,52,366.0,26.0,101.4,106.4,-4.84,1991,WSB
4,26,56,317.0,30.0,102.9,107.5,-4.53,1991,NJN
...,...,...,...,...,...,...,...,...,...
1028,42,30,583.0,NaN,112.4,110.2,2.26,2021,DAL
1029,38,34,528.0,4.0,113.3,112.3,1.07,2021,MEM
1030,33,39,458.0,9.0,111.1,112.8,-1.58,2021,SAS
1031,31,41,431.0,11.0,114.6,114.9,-0.20,2021,NOP


In [148]:
from datetime import datetime


def cleaning_inflationDf_separateDateColumns(us_inflation_2b, column='Yearmon', format="%d-%m-%Y"):
    us_inflation_2b = us_inflation_2b.copy()

    dates = []
    for i in range(len(us_inflation_2b)):
        date = us_inflation_2b[column].iloc[i]
        date = datetime.strptime(date, format)
        dates.append(date)
    us_inflation_2b['Year']  = [date.year for date in dates]
    us_inflation_2b['Month'] = [date.month for date in dates]
    us_inflation_2b['Day']   = [date.day for date in dates]

    return us_inflation_2b


def cleaning_inflationDf_getYearByMedian(us_inflation_2b):
    """
    precondition: cleaning_inflationDf_separateDateColumns
    """
    us_inflation_2b = us_inflation_2b.copy()
    us_inflation_2b = cleaning_inflationDf_separateDateColumns(us_inflation_2b)
    us_inflation_2b = us_inflation_2b.groupby(by='Year')['CPI'].median()
    us_inflation_2b = us_inflation_2b.reset_index()
    return us_inflation_2b


def cleaning_anyDf_removeDuplicatedColumns(combined_df):
    combined_df = combined_df.copy()
    columns = combined_df.columns

    columns_tobe_removed = []
    for i in range(len(columns)):
        col1 = columns[i]
        for j in range(i+1, len(columns)):
            col2 = columns[j]

            if (combined_df[col1] == combined_df[col2]).all():
                columns_tobe_removed.append(col1)
    columns_tobe_kept = set(combined_df.columns) - set(columns_tobe_removed)
    columns_tobe_kept = list(columns_tobe_kept)
    return combined_df.loc[:, columns_tobe_kept]

def cleaning_anyDf_recalculateWLpercentage(teams_2b):
    teams_2b = teams_2b.copy()
    teams_2b['W/L%'] = 100*teams_2b.W / (teams_2b.W + teams_2b.L)
    return teams_2b

In [149]:
us_inflation_2b.head()

,Yearmon,CPI
0,01-01-1913,9.8
1,01-02-1913,9.8
2,01-03-1913,9.8
3,01-04-1913,9.8
4,01-05-1913,9.7


In [150]:
cleaning_inflationDf_getYearByMedian(cleaning_inflationDf_separateDateColumns(us_inflation_2b))

,Year,CPI
0,1913,9.8500
1,1914,10.0000
2,1915,10.1000
3,1916,10.8000
4,1917,12.9000
...,...,...
104,2017,244.8705
105,2018,251.7885
106,2019,256.3505
107,2020,258.8895


Pada kasus ini kami memutuskan mengambil median dari inflasi karena CPI merupakan perbandingan inflasi pada tahun ini dengan inflasi pada suatu tahun yang dijadikan pivot. Karena CPI merupakan suatu perbandingan, maka kami merasa bahwa mengambil median lebih tepat dibandingkan mengambil mean-nya.

In [151]:
us_inflation_2b_cleaned = us_inflation_2b.copy()
us_inflation_2b_cleaned = cleaning_inflationDf_separateDateColumns(us_inflation_2b_cleaned)
us_inflation_2b_cleaned = cleaning_inflationDf_getYearByMedian(us_inflation_2b_cleaned)

In [152]:
combined_df = us_inflation_2b_cleaned.merge(teams_2b, left_on=["Year"], right_on=["Year"])
combined_df = cleaning_anyDf_recalculateWLpercentage(combined_df)
combined_df.head(3)

,Year,CPI,W,L,W/L%,GB,PS/G,PA/G,SRS,Team
0,1991,136.1,56,26,68.292683,NaN,111.5,105.7,5.22,BOS
1,1991,136.1,44,38,53.658537,12.0,105.4,105.6,-0.39,PHI
2,1991,136.1,39,43,47.560976,17.0,103.1,103.3,-0.43,NYK


In [153]:
mvp_stats_2b = mvp_stats_2b.drop('Tm', axis=1)  # karena sudah ada kolom Team
mvp_stats_2b = cleaning_anyDf_convertTeamNamesToItsAbbrev(mvp_stats_2b, nicknames_df)

In [154]:
display(mvp_stats_2b.head(2))
display(combined_df.head(2))
print('banyaknya null salaries: ', salaries_2b.Salaries.isna().sum())
print('banyaknya null player name salaries: ', salaries_2b.Name.isna().sum())
print('banyaknya null player name mvp stats: ', mvp_stats_2b.Player.isna().sum())
print('banyaknya null team name: ', combined_df.Team.isna().sum())
print()
print('banyaknya row salaries df : ', len(salaries_2b))
print('banyaknya row mvp statsdf : ', len(mvp_stats_2b))
print('banyaknya row combined df : ', len(combined_df))

,Player,Pos,Age,G,GS,MP,FG,FGA,FG%,3P,...,Pts Max,Share,Team,W,L,W/L%,GB,PS/G,PA/G,SRS
0,A.C. Green,PF,27,82,21,26.4,3.1,6.6,476.0,0.1,...,0,0.0,LAL,58,24,707.0,5.0,106.3,99.6,6.73
1,Byron Scott,SG,29,82,82,32.1,6.1,12.8,477.0,0.9,...,0,0.0,LAL,58,24,707.0,5.0,106.3,99.6,6.73


,Year,CPI,W,L,W/L%,GB,PS/G,PA/G,SRS,Team
0,1991,136.1,56,26,68.292683,NaN,111.5,105.7,5.22,BOS
1,1991,136.1,44,38,53.658537,12.0,105.4,105.6,-0.39,PHI


banyaknya null salaries:  28074
banyaknya null player name salaries:  0
banyaknya null player name mvp stats:  0
banyaknya null team name:  0

banyaknya row salaries df :  37420
banyaknya row mvp statsdf :  14092
banyaknya row combined df :  906


In [155]:
salaries_2b = salaries_2b[["Name", "Year", "Salaries", "Rank"]]

combined_df = mvp_stats_2b.merge(combined_df, left_on=["Team", "Year"], right_on=["Team", "Year"])
combined_df = salaries_2b.merge(combined_df, left_on=["Name", "Year"], right_on=["Player", "Year"])
combined_df = cleaning_anyDf_removeDuplicatedColumns(combined_df)

combined_df.head(5)

,FG%,W_y,G,GB_y,AST,Rank,MP,SRS_y,TRB,Salaries,...,L_y,W/L%_y,Player,FT%,FT,CPI,PS/G_y,3P%,Pts Won,PTS
0,574.0,67,79,NaN,3.8,1,40.0,8.41,13.6,17142000.0,...,15,81.707317,Shaquille O'Neal,524.0,5.5,172.6,100.8,0.00,1207,29.7
1,497.0,50,81,5.0,5.0,2,40.0,2.67,11.8,16806000.0,...,32,60.975610,Kevin Garnett,765.0,3.8,172.6,98.5,0.37,408,22.9
2,551.0,52,79,NaN,1.6,3,34.8,2.75,9.5,15004000.0,...,30,63.414634,Alonzo Mourning,711.0,5.2,172.6,94.4,0.00,367,21.7
3,459.0,29,82,23.0,3.0,4,35.5,-3.47,5.7,15000000.0,...,53,35.365854,Juwan Howard,735.0,2.5,172.6,96.6,0.00,0,14.9
4,451.0,59,82,8.0,5.0,5,33.5,6.36,6.3,14795000.0,...,23,71.951220,Scottie Pippen,717.0,2.0,172.6,97.5,327.00,0,12.5


In [156]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8323 entries, 0 to 8322
Data columns (total 45 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   FG%       8296 non-null   float64
 1   W_y       8323 non-null   int64  
 2   G         8323 non-null   int64  
 3   GB_y      6705 non-null   float64
 4   AST       8323 non-null   float64
 5   Rank      8323 non-null   int64  
 6   MP        8323 non-null   float64
 7   SRS_y     8323 non-null   float64
 8   TRB       8323 non-null   float64
 9   Salaries  7933 non-null   float64
 10  2P%       8276 non-null   float64
 11  Age       8323 non-null   int64  
 12  BLK       8323 non-null   float64
 13  GS        8323 non-null   int64  
 14  eFG%      8296 non-null   float64
 15  Team      8323 non-null   object 
 16  STL       8323 non-null   float64
 17  2P        8323 non-null   float64
 18  TOV       8323 non-null   float64
 19  ORB       8323 non-null   float64
 20  FTA       8323 non-null   floa

### 2B. Drop missing values pada target feature

kami memutuskan untuk mendrop missing values pada target feature karena target feature merupakan ground truth. Oleh karena itu, kami tidak setuju apabila kita melakukan imputasi untuk missing values pada kolom ini. Sebab, apabila kita melakukan imputasi terhadap missing values untuk kolom ini, maka kolom ini tidak sepenuhnya menjadi ground truth lagi karena diisikan dengan data-data sintesis yang tidak diketahui kebenarannya.

In [157]:
print("Banyak missing values pada kolom Salaries: ", combined_df.Salaries.isna().sum())
combined_df = combined_df[combined_df.Salaries.notna()]
combined_df.isna().sum()

Banyak missing values pada kolom Salaries:  390


FG%           25
W_y            0
G              0
GB_y        1571
AST            0
Rank           0
MP             0
SRS_y          0
TRB            0
Salaries       0
2P%           43
Age            0
BLK            0
GS             0
eFG%          25
Team           0
STL            0
2P             0
TOV            0
ORB            0
FTA            0
GB_x           0
FGA            0
PF             0
3PA            0
Year           0
3P             0
DRB            0
PA/G_y         0
Pts Max        0
FG             0
W/L%_x         0
Pos            0
2PA            0
Share          0
L_y            0
W/L%_y         0
Player         0
FT%          230
FT             0
CPI            0
PS/G_y         0
3P%         1117
Pts Won        0
PTS            0
dtype: int64

### 2B. Memisahkan kolom-kolom menjadi feature (X) dan target (y) dan drop fitur-fitur yang tidak diperlukan

Pada dataset ini, kita sudah memiliki gabungan seluruh feature mengenai statistik performa team dan statistik performa pemain. Oleh karena itu, nama tim dan nama pemain sudah tidak lagi berkaitan dengan gaji yang diperoleh. Nama tim sudah dapat digantikan dengan data mengenai performa tim, dan nama pemain dapat digantikan dengan data mengenai performa pemain.

Sementara itu, kita melakukan drop kolom Rank karena kolom tersebut merupakan Ranking gaji seorang pemain pada tahun itu. Oleh karena itu, kami merasa informasi ini tidak valid untuk dijadikan dasar dalam memprediksi gaji pemain.

In [158]:
x_2b, y_2b = combined_df.drop("Salaries", axis=1), combined_df[["Salaries"]]
x_2b.drop("Team", axis=1, inplace=True)
x_2b.drop("Player", axis=1, inplace=True)
x_2b.drop("Rank", axis=1, inplace=True)

display(x_2b.head(2))
display(y_2b.head(2))

,FG%,W_y,G,GB_y,AST,MP,SRS_y,TRB,2P%,Age,...,Share,L_y,W/L%_y,FT%,FT,CPI,PS/G_y,3P%,Pts Won,PTS
0,574.0,67,79,NaN,3.8,40.0,8.41,13.6,575.0,27,...,998.0,15,81.707317,524.0,5.5,172.6,100.8,0.00,1207,29.7
1,497.0,50,81,5.0,5.0,40.0,2.67,11.8,504.0,23,...,337.0,32,60.975610,765.0,3.8,172.6,98.5,0.37,408,22.9


,Salaries
0,17142000.0
1,16806000.0


### 2B. Membagi dataset menjadi training, testing, dan final testing

Kami memutuskan untuk membagi dataset menjadi training, testing, dan final testing. Dataset final testing diperlukan karena kita sangat disarankan untuk memasuki fase testing hanya sekali saja, yakni ketika kita sudah benar-benar yakin dengan model yang kita bangun dan sudah tidak ingin mengutak-atik modelnya lagi. Oleh karena itu, kami memutuskan untuk membagi dataset testing menjadi testing untuk mengecek performa, dan final testing untuk model yang sudah final.

In [159]:
x_2b_train, x2b_final_test, y_2b_train, y_2b_final_test = train_test_split(
    x_2b, y_2b, test_size=0.2, random_state=random_state
)
del x_2b
del y_2b

x_2b_train, x2b_test, y_2b_train, y_2b_test = train_test_split(
    x_2b_train, y_2b_train, test_size=0.2, random_state=random_state,
)

### 2B training model

In [160]:
class Alternative(TransformerMixin):
    def __init__(self, *models, curr_model=0):
        self.__curr_model = curr_model
        self.models = models

    def fit(self, X, y=None):
        self.get_curr_model().fit(X, y)
        return self

    def predict(self, X):
        return self.get_curr_model().predict(X)

    def transform(self, X):
        res = self.get_curr_model().transform(X)
        return res

    def set_curr_model(self, model_index: int):
        self.__curr_model = model_index

    def get_curr_model(self):
        return self.models[self.__curr_model]

    def set_params(self, **kwargs):
        if 'curr_model' in kwargs:
            self.set_curr_model(kwargs['curr_model'])
            del kwargs['curr_model']
        self.get_curr_model().set_params(**kwargs)
        return self

def cartesian_product_of_dict_lists(*multiple_list_of_dicts):
    if len(multiple_list_of_dicts) == 0:
        return []

    if len(multiple_list_of_dicts) == 1:
        return multiple_list_of_dicts[0]

    if len(multiple_list_of_dicts) == 2:
        ret = []
        for dict1 in multiple_list_of_dicts[0]:
            for dict2 in multiple_list_of_dicts[1]:
                ret.append(dict(**dict1, **dict2))
        return ret

    curr = [{}]
    for dct in multiple_list_of_dicts:
        curr = cartesian_product_of_dict_lists(curr, dct)
    return curr

In [161]:
from sklearn.base import TransformerMixin #gives fit_transform method for free

# LabelEncoder tidak dapat dimasukkan ke dalam pipeline karena
# LabelEncoder.transform() tidak memiliki parameter y

class MyLabelEncoder(TransformerMixin):
    # source: https://stackoverflow.com/a/46619402/7069108
    def __init__(self, columns_to_be_encoded: list[str], *args, **kwargs):
        self.columns_to_be_encoded = columns_to_be_encoded
        self.encoder = preprocessing.LabelEncoder(*args, **kwargs)

    def fit(self, x, y=0):
        for col in self.columns_to_be_encoded:
            self.encoder.fit(x[col])
        return self

    def transform(self, x, y=0):
        x = x.copy()
        for col in self.columns_to_be_encoded:
            x[col] = self.encoder.transform(x[col])
        return x

label_encoder = MyLabelEncoder(["Pos"])
label_encoder_params = [{}]

In [162]:
scaler1 = preprocessing.MinMaxScaler()
scaler2 = preprocessing.StandardScaler()
scaler3 = preprocessing.RobustScaler()

scaler = Alternative(
    scaler1, scaler2, scaler3
)

scaler_params = [{
    'scaler__curr_model': [0, 1, 2]
}]

In [163]:
imputer1 = SimpleImputer()
imputer2 = KNNImputer()

imputer = Alternative(
    imputer1, imputer2,
)

imputer_params = [{
    'imputer__curr_model': [0, 1]
}]

In [164]:
feature_selector1 = SelectFromModel(estimator=Lasso())
feature_selector2 = SelectKBest()

feature_selector = Alternative(
    feature_selector2,
    feature_selector1
)

feature_selector_params = [{
    'feature_selector__curr_model': [1],
    'feature_selector__estimator__alpha': [0.01, 0.005, 0.0005]
}, {
    'feature_selector__curr_model': [0],
    'feature_selector__k': [7, 8, 9],
}]

In [165]:
predictors  = Alternative(
    LinearRegression(),
    RandomForestRegressor(),
)

predictor_params = [{
    'predictor__curr_model': [0, 1]
}]

In [166]:
pipeline = Pipeline(
    steps=[
        ('encoding', label_encoder),
        ('scaler', scaler),
        ('imputer', imputer),
        ('feature_selector', feature_selector),
        ('predictor', predictors),
    ]
)

In [167]:
grid_params = cartesian_product_of_dict_lists(
    label_encoder_params,
    scaler_params,
    imputer_params,
    feature_selector_params,
    predictor_params
)

Pipeline(steps=[('encoding',
                 <__main__.MyLabelEncoder object at 0x00000285C8414430>),
                ('scaler', <__main__.Alternative object at 0x00000285C7F889D0>),
                ('imputer',
                 <__main__.Alternative object at 0x00000285C8415060>),
                ('feature_selector',
                 <__main__.Alternative object at 0x00000285C7E77DF0>),
                ('predictor',
                 <__main__.Alternative object at 0x00000285C8417FA0>)])

In [169]:
grid_search_cv = GridSearchCV(
    estimator=pipeline,
    param_grid=grid_params,
    scoring='neg_mean_absolute_error',
    n_jobs=-1,
)

grid_search_cv.fit(x_2b_train, y_2b_train.Salaries)

H:\01 Kuliah\01 Dokumen\52 - KASDD\proyek-akhir\venv\lib\site-packages\sklearn\model_selection\_search.py:953: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(
H:\01 Kuliah\01 Dokumen\52 - KASDD\proyek-akhir\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.139e+16, tolerance: 1.397e+13
  model = cd_fast.enet_coordinate_descent(


GridSearchCV(estimator=Pipeline(steps=[('encoding',
                                        <__main__.MyLabelEncoder object at 0x00000285C8414430>),
                                       ('scaler',
                                        <__main__.Alternative object at 0x00000285C7F889D0>),
                                       ('imputer',
                                        <__main__.Alternative object at 0x00000285C8415060>),
                                       ('feature_selector',
                                        <__main__.Alternative object at 0x00000285C7E77DF0>),
                                       ('predictor',
                                        <__main__.Alternative object at 0x00000285C84...
             param_grid=[{'feature_selector__curr_model': [1],
                          'feature_selector__estimator__alpha': [0.01, 0.005,
                                                                 0.0005],
                          'imputer__curr_model': [0, 1],
                          'predictor__curr_model': [0, 1],
                          'scaler__curr_model': [0, 1, 2]},
                         {'feature_selector__curr_model': [0],
                          'feature_selector__k': [7, 8, 9],
                          'imputer__curr_model': [0, 1],
                          'predictor__curr_model': [0, 1],
                          'scaler__curr_model': [0, 1, 2]}],
             scoring='neg_mean_absolute_error')

['grid_search_cv_2b.pkl']

### 2B testing

In [174]:
y_2b_predict = grid_search_cv.predict(x2b_test)
mae = metrics.mean_absolute_error(y_2b_test, y_2b_predict)
print("Rata-rata error absolut: ", mae)
print("Rata-rata error relatif: ", mae / y_2b_test.Salaries.mean())

Rata-rata error absolut:  2502435.804846689
Rata-rata error relatif:  0.5432521005553794


In [172]:
grid_search_cv.best_params_

{'feature_selector__curr_model': 1,
 'feature_selector__estimator__alpha': 0.01,
 'imputer__curr_model': 0,
 'predictor__curr_model': 0,
 'scaler__curr_model': 0}

In [173]:
grid_search_cv.best_estimator_['feature_selector'].models[0].get_feature_names_out(input_features=x_2b_train.columns)

array(['Age', '2P', 'FTA', 'FGA', 'Year', 'FG', '2PA', 'FT', 'CPI', 'PTS'],
      dtype=object)